In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv
/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip
/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip


In [2]:
train=pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip')
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [4]:
train['Phrase'].iloc[0]

'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'

In [5]:
test=pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip')
test


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


In [9]:
all_data=pd.concat([train,test])

all_data['Phrase']=all_data['Phrase'].fillna('')

from tensorflow.keras.preprocessing.text import Tokenizer
tk=Tokenizer()

#등록 fitting/전처리 
tk.fit_on_texts(all_data['Phrase'])

In [14]:
#tk.word_index
#빈도가 높으면 순번이 높다 
#tk.word_counts

TypeError: unhashable type: 'slice'

In [19]:
#transform 과정 
text=tk.texts_to_sequences(all_data['Phrase'])
print(text[0])

[2, 315, 3, 16573, 7660, 1, 8313, 9, 53, 8, 47, 13, 1, 3940, 8, 187, 47, 13, 1, 13024, 61, 3, 89, 592, 12156, 19, 622, 3, 89, 2810, 5, 52, 3, 2, 42]


In [22]:
#text 사이즈 일반화 시키기 
from tensorflow.keras.preprocessing.sequence import pad_sequences
#text 를 넣어야한다 이미 전처리가 되어있기 때문에 
all_pad=pad_sequences(text)
all_pad.shape

(222352, 52)

In [24]:
#모델링
train_2=all_pad[:len(train)]
test_2=all_pad[len(train):]
train_2


array([[    0,     0,     0, ...,     3,     2,    42],
       [    0,     0,     0, ...,    13,     1,  3940],
       [    0,     0,     0, ...,     0,     2,   315],
       ...,
       [    0,     0,     0, ...,     0, 11171, 11172],
       [    0,     0,     0, ...,     0,     0, 11171],
       [    0,     0,     0, ...,     0,     0, 11172]], dtype=int32)

In [25]:
len(tk.word_index)

17780

In [28]:
from tensorflow.keras import *
from tensorflow.keras.layers import *

model=Sequential()
#embedding 에서 단어의 뜻을 알아내는 역할을 하는 층//단어+1 0 부터 시작하기때문에//17781 옆에 적은 숫자는 하나의 단어를 학습할때 몇 차원에 vector 로 학습할것인가 (예를 들어 사과라는 단어의 뜻을 1개만 할건지 더 자세히 할것인지//각각의 색깔, 모양 등등 ) 
#하나의 데이터에 차원넣기 세번째옵션 (하나의 데이터에 몇단어씩 이루어져있는지)
model.add(Embedding(17781,1))
model.add(Flatten())
model.add(Dense(5,activation='softmax'))
#loss 가 바뀐다 image generator 에서 바뀐다 분류가 숫자적인걸로 영향을 받는건지 몰라서 
model.compile(metrics=['acc'], optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(train_2,train['Sentiment'],epochs=3)

Epoch 1/3
4877/4877 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - acc: 0.5114 - loss: 1.2326
Epoch 2/3
4877/4877 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - acc: 0.5595 - loss: 1.0921
Epoch 3/3
4877/4877 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - acc: 0.5766 - loss: 1.0515


In [29]:
#predict 에서는 무조건확률로 나온다 딥러닝 
result=model.predict(test_2)
result

2072/2072 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


array([[0.0721858 , 0.28016105, 0.36576828, 0.23240893, 0.04947594],
       [0.0721858 , 0.28016105, 0.36576828, 0.23240893, 0.04947594],
       [0.01676957, 0.13617791, 0.7420533 , 0.09887467, 0.00612451],
       ...,
       [0.02542159, 0.16330788, 0.6425397 , 0.15550317, 0.01322768],
       [0.02542159, 0.16330788, 0.6425397 , 0.15550317, 0.01322768],
       [0.01688761, 0.13575275, 0.7239043 , 0.11558569, 0.00786961]],
      dtype=float32)

In [31]:
sub=pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv')
sub

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
...,...,...
66287,222348,2
66288,222349,2
66289,222350,2
66290,222351,2


In [32]:
sub['Sentiment']=result.argmax(1)
sub

,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2
...,...,...
66287,222348,2
66288,222349,2
66289,222350,2
66290,222351,2


In [34]:
sub.to_csv('sub789.csv',index=0)